<a href="https://colab.research.google.com/github/clive-limo/DataScienceProjects/blob/main/Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hate Speech Detection

---



The dataset used in this paper is a dynamically generated hate speech dataset from Github based on a paper by [Vidgen et al. (2020)](https://arxiv.org/abs/2012.15761). 

> The dataset has a total 40,623 examples of hate speech and no hate speech text including fine-grained labels and a large number of challenging contrastive perturbation examples. It comprises 54% hateful and 46% not hateful entries.


# Setting up the Environment

#### Install Dependencies



In [ ]:
!pip install -q transformers
!pip install -q simpletransformers

     |████████████████████████████████| 5.5 MB 5.3 MB/s 
     |████████████████████████████████| 163 kB 62.1 MB/s 
     |████████████████████████████████| 7.6 MB 55.1 MB/s 
     |████████████████████████████████| 250 kB 5.1 MB/s 
     |████████████████████████████████| 1.3 MB 48.4 MB/s 
     |████████████████████████████████| 9.2 MB 32.9 MB/s 
     |████████████████████████████████| 1.9 MB 47.5 MB/s 
     |████████████████████████████████| 441 kB 58.2 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 71.6 MB/s 
     |████████████████████████████████| 182 kB 71.7 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 166 kB 71.1 MB/s 
     |████████████████████████████████| 162 kB 75.2 MB/s 
     |████████████████████████████████| 162 kB 67.9 MB/s 
     |████████████████████████████████| 158 kB 81.4 MB/s 
     |████████████████████████████████| 157 kB 70.3 MB/s 
     |██████████████

# Importing Data

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset/main/Dynamically%20Generated%20Hate%20Dataset%20v0.2.3.csv')
# split the dataset by the column 'split' which labels 'train' and 'test' samples
train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

In [ ]:
train_df.head()

,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
0,1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
14,15,acl1001,15,It's such a rape culture with these immigrants...,hate,notgiven,notgiven,original,train,1,CAgNlUizNm,1,NaN


### Pre Processing

Transform the values in the label column from 'hate' and 'nothate' to 1 and 0 respectively. This is required in order for us to be able to train the model properly.

In [ ]:
train_df = train_df.replace({'label': {'hate': 1, 'nothate': 0}}) # relabel the `label` column, hate is 1 and nothate is 0
test_df = test_df.replace({'label': {'hate': 1, 'nothate': 0}}) # relabel the `label` column, hate is 1 and nothate is 0
train_df.head()

,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
0,1,acl1,1,dalits are lowlives,1,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,2,acl10,2,gay people are a burden to our society,1,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,3,acl100,3,Arabs are not welcome,1,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,4,acl1000,4,I'm not saying we should actually eliminate he...,1,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
14,15,acl1001,15,It's such a rape culture with these immigrants...,1,notgiven,notgiven,original,train,1,CAgNlUizNm,1,NaN


Rename the `label` column to `labels` as is required by the `simpletransformers` library.

In [ ]:
train_df = train_df.rename(columns={'label': 'labels'})
test_df = test_df.rename(columns={'label': 'labels'})

Check shape of the datasets

In [ ]:
data = [[train_df.labels.value_counts()[0], test_df.labels.value_counts()[0]], 
        [train_df.labels.value_counts()[1], test_df.labels.value_counts()[1]]]
# Prints out the dataset sizes of train test and validate as per the table.
pd.DataFrame(data, columns=['Train', 'Test'])

,Train,Test
0,15184,1852
1,17740,2268


# Training and Testing the Model

#### Setting up hyperparmeters

In [ ]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 1,
    'train_batch_size': 128,
    'fp16': True,
    'output_dir': '/outputs/',
}

#### Train the Model


In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the XLNet base cased pre-trained model.
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=train_args) 

# Train the model, there is no development or validation set for this dataset
model.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/298 [00:00<?, ?it/s]

  0%|          | 0/4120 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/587 [00:00<?, ?it/s]

The model has been trained and evaluating on the test set after training to only 1 epoch gives an accuracy of **85.9%**. We want to also evaluate the F1 score which is a better measure as the dataset is slightly imbalanced.

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.f1_score)

  0%|          | 0/4120 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/587 [00:00<?, ?it/s]

We see that the output F1 score from the model after training for 1 epoch is **86.6%** ('acc': 0.8661675245671503).

# Testing the model

Predictions using the model and random samples from the testing dataset

In [ ]:
samples = [test_df[test_df['labels'] == 0].sample(10).iloc[0]['text']] # get a random sample from the test set which is nothate
predictions, _ = model.predict(samples)
label_dict = {0: 'nothate', 1: 'hate'}
for idx, sample in enumerate(samples):
  print('{} - {}: {}'.format(idx, label_dict[predictions[idx]], sample))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 - nothate: left wingers have become so inane, their ideologies should not be part of the respectable part of this world.


We can also generate a `results.txt` file from the test set. In the `result.txt` file `0` indicates `not hate` while `1` indicates `hate speech`

In [ ]:
predictions, _ = model.predict(test_df['text'].tolist())

df = pd.DataFrame(predictions)
df['text']=test_df['text'].tolist()
df.to_csv('results.txt', index=False, header=False)

  0%|          | 0/4120 [00:00<?, ?it/s]

  0%|          | 0/587 [00:00<?, ?it/s]